This notebook tests various LLMs in upselling classification.

Notes:
- I did not use a diarized transcript, but I suspect using one would be better.

To-do:
- Add more model results
- Test with diarized transcript
- Test on upsizing and customer loyalty classification?
- Double-check model licenses

## Set-up

Installs were not added to `requirements.txt` as I ran this notebook in Google Colab did not check for conflicts with repo dependencies.

In [ ]:
# !pip install transformers accelerate einops

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Llama-2

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
prompt = '''Thank you for choosing to be here. How can I help you? I'm going to do a turtle, peacock, cluster, blizzard. What size? What sizes do you have? Mini, small, medium, and large. Medium. Okay, can I get you anything else? This is for you. The ultimate color brownie. Medium as well? Medium as well. That'll be it. I think that's going to be 15 cent out the window. Thank you. Thank you.

Did upselling happen in this conversation?
'''

torch.manual_seed(42)
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
    return_full_text=False
)

In [ ]:
for seq in sequences:
  print(seq['generated_text'])

`Yes, upselling happened in this conversation. The cashier suggested larger sizes of the products (clusters, blizzards) and offered a more expensive option (the ultimate color brownie) even though the customer only asked for a medium-sized item.`

## Falcon

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
  torch_dtype=torch.bfloat16,
  trust_remote_code=True,
  device_map="auto",
)

In [ ]:
prompt = '''Thank you for choosing to be here. How can I help you? I'm going to do a turtle, peacock, cluster, blizzard. What size? What sizes do you have? Mini, small, medium, and large. Medium. Okay, can I get you anything else? This is for you. The ultimate color brownie. Medium as well? Medium as well. That'll be it. I think that's going to be 15 cent out the window. Thank you. Thank you.

Did upselling happen in this conversation?
'''

torch.manual_seed(42)
sequences = pipeline(
  prompt,
  do_sample=True,
  top_k=10,
  num_return_sequences=1,
  eos_token_id=tokenizer.eos_token_id,
  max_length=200,
  return_full_text=False
)

In [ ]:
for seq in sequences:
  print(seq['generated_text'])

`Yes, it appears that upselling occurred with the offer of additional items and packages for the customer to purchase during the initial offer.`